# renaming the reactions according to columns and rows

In [34]:
from math import isclose
from json import load, dump

with open("WesternDietMicrobiomes/escherMap.json", "r") as jsonIn:    escherMap = load(jsonIn)

# display(escherMap)

xs, ys = [], []
for index, lst in enumerate(escherMap):
    for k, content in lst.items():
        if k != "reactions":  continue
        print(content)
        for rxn, content2 in content.items():
            if "label_x" not in content2:  print(content2) ; continue
            if not any([isclose(content2["label_x"], x, abs_tol=50) for x in xs]):   xs.append(content2["label_x"])
            if not any([isclose(content2["label_y"], y, abs_tol=50) for y in ys]):   ys.append(content2["label_y"])

print(xs, ys)

for index, lst in enumerate(escherMap):
    for k, content in lst.items():
        if k != "reactions":  continue
        for rxn, content2 in content.items():
            for x in xs:
                if isclose(content2["label_x"], x, abs_tol=50):
                    col_index = xs.index(x)
                    continue
            for y in ys:
                if isclose(content2["label_y"], y, abs_tol=50):
                    row_index = ys.index(y)
                    continue
            for attribute, val in content2.items():
                if attribute == "metabolites":
                    metabolite_dics = []
                    for metabolite in val:
                        metabolite_dic = {}
                        for cpd_index, (k2, val2) in enumerate(metabolite.items()):
                            if k2 == "bigg_id":  metabolite_dic[k2] = f"{val2}_{col_index}_{row_index}_{cpd_index}"
                            elif k2 == "coefficient":  metabolite_dic[k2] = val2
                        metabolite_dics.append(metabolite_dic)
                    escherMap[index][k][rxn][attribute] = metabolite_dics
                    print(escherMap[index][k][rxn][attribute])
                if attribute == "bigg_id":   escherMap[index][k][rxn][attribute] = f"{val}_{col_index}_{row_index}"
                # if attribute == "metabolites":
                    # escherMap[index][k][rxn][attribute] = val+
                # print(attribute, val)


with open("WesternDietMicrobiomes/escherMap_edited.json", "w") as jsonOut:    escherMap = dump(escherMap, jsonOut, indent=3)

{'0': {'name': 'diphosphate phosphohydrolase', 'bigg_id': 'rxn00001', 'reversibility': False, 'label_x': -642.8153489006918, 'label_y': 103.11225355934926, 'gene_reaction_rule': '', 'genes': [], 'metabolites': [{'bigg_id': 'cpd00012', 'coefficient': -1}, {'bigg_id': 'cpd00067', 'coefficient': 1}, {'bigg_id': 'cpd00001', 'coefficient': -1}, {'bigg_id': 'cpd00009', 'coefficient': 2}], 'segments': {'0': {'from_node_id': '1', 'to_node_id': '2', 'b1': None, 'b2': None}, '1': {'from_node_id': '3', 'to_node_id': '2', 'b1': None, 'b2': None}, '2': {'from_node_id': '1', 'to_node_id': '0', 'b1': {'x': -638.8122011531899, 'y': 126.74002323747482}, 'b2': {'x': -693.0594201712433, 'y': 127.28932170583562}}, '3': {'from_node_id': '3', 'to_node_id': '4', 'b1': {'x': -514.8185576833539, 'y': 125.48448388122324}, 'b2': {'x': -460.57133866530046, 'y': 124.93518541286244}}, '4': {'from_node_id': '1', 'to_node_id': '5', 'b1': {'x': -642.8119961038293, 'y': 126.78052450703285}, 'b2': {'x': -647.87947126354

# duplicating reactions across a specified number of rows and columns

In [45]:
2961.18779884681--638.8122011531899

3600.0

In [38]:
from json import load, dump

def duplicate_rxn(escherJSON_path, rows=None, columns=None, row_step_size=800, col_step_size=400):
    assert rows is not None or columns is not None, "What are you doing?"

    with open(escherJSON_path, "r") as jsonIn:
        escherMap = load(jsonIn)

    # display(escherMap)
    print(len(escherMap))

    print(row_step_size, col_step_size)

    for i, lst in enumerate(escherMap):
        if i == 0:  continue
        for k, content in lst.items():
            if k == "reactions":
                # row reaction development
                print("rows")
                print(row_step_size, col_step_size)
                updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                if isinstance(rows, int):
                    row_reactions = {k:v for k,v in content.items()}
                    for row_index in range(1, rows):
                        # print(content.keys())
                        for rxnNum, content2 in content.items():
                            nextRxn = str(int(rxnNum)+1)
                            ## define the content of this reaction
                            new_content = {k:v for k,v in content2.items()}
                            ## translate the label
                            new_content["label_y"] += row_step_size*row_index
                            ## rename the reaction according to its new translation
                            rxnName_content = new_content["bigg_id"].split("_")
                            colNum, rowNum = rxnName_content[1], rxnName_content[2]
                            new_content["bigg_id"] = f"{rxnName_content[0]}_{colNum}_{str(int(rowNum)+1)}"
                            ## rename the metabolites
                            for index, metabolite in enumerate(new_content["metabolites"]):
                                ### rename the metabolites
                                ID_content = metabolite["bigg_id"].split("_")
                                metabolite["bigg_id"] = f"{ID_content[0]}_{colNum}_{rowNum}_{index}"
                            ## update the metabolite nodes
                            display(new_content.keys())
                            ## update the edge segments
                            for num, content3 in new_content["segments"].items():
                                dictionary_values = [value for value in content3.values() if isinstance(value, dict)]
                                # print(dictionary_values)
                                # for obj, value in content3.items():
                                #     # print(list(map(len, [new_content, content3])))
                                #     if isinstance(value, dict):
                                #         print(rxnNum, num, obj, value) ;  new_content["segments"][num][obj]["y"] += row_step_size*row_index
                                #     if isinstance(value, dict):
                                #         print(new_content["segments"][num][obj])
                            row_reactions[str(int(rxnNum)+row_index)] = new_content
                            # print(row_reactions[str(int(rxn)+row_index)]["segments"]["312"]["b2"])
                        updated_reactions.update({k:v for k,v in row_reactions.items()})
            elif k == "nodes":
                # row metabolite development
                print("rows")
                print(row_step_size, col_step_size)
                updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                if isinstance(rows, int):
                    row_nodes = {k:v for k,v in content.items()}
                    for row_index in range(1, rows):
                        print(content.keys())
                        # updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                        new_mets = {}
                        for metNum, content2 in row_nodes.items():
                            ### rename the node for this set of reactions
                            # print(metNum)
                            new_mets[f"{nextRxn}{metNum}"] = content2
                            ### translating the node locations
                            if new_mets[f"{nextRxn}{metNum}"]["node_type"] in ["multimarker", "midmarker"]:
                                new_mets[f"{nextRxn}{metNum}"]["y"] += row_step_size*row_index
                            elif new_mets[f"{nextRxn}{metNum}"]["node_type"] == "metabolite":
                                new_mets[f"{nextRxn}{metNum}"]["y"] += row_step_size*row_index
                                new_mets[f"{nextRxn}{metNum}"]["label_y"] += row_step_size*row_index
                        updated_reactions.update({k:v for k,v in row_reactions.items()})
            # else:   print(k)
        escherMap[i][k] = updated_reactions  
        # if index == len(escherMap)-1:  break
        # print("columns")
        # print(row_step_size, col_step_size)
        # if isinstance(columns, int):
        #     reactions = content if rows is not None else updated_reactions
        #     col_reactions = {}
        #     for col_index in range(columns):
        #         for rxnNum, content2 in reactions.items():
        #             new_content = {k:v for k,v in content2.items()}
        #             new_content["label_x"] += col_step_size*col_index
        #             for num, content3 in new_content["segments"].items():
        #                 for obj, value in content3.items():
        #                     if isinstance(value, dict):  print(rxnNum, num, obj, value) ; new_content["segments"][num][obj]["x"] += col_step_size*col_index
        #                     if isinstance(value, dict):  print(new_content["segments"][num][obj])
        #             col_reactions[str(int(rxnNum)+col_index)] = new_content
        #         updated_reactions.update({k:v for k,v in row_reactions.items()})

        #     escherMap[index][k] = updated_reactions    

    with open(escherJSON_path.replace(".json", "_edited.json"), "w") as jsonOut:  dump(escherMap, jsonOut, indent=3)

duplicate_rxn("oneRXN_localized2.json", rows=3, columns=None)

2
800 400
rows
800 400


dict_keys(['name', 'bigg_id', 'reversibility', 'label_x', 'label_y', 'gene_reaction_rule', 'genes', 'metabolites', 'segments'])

dict_keys(['name', 'bigg_id', 'reversibility', 'label_x', 'label_y', 'gene_reaction_rule', 'genes', 'metabolites', 'segments'])

rows
800 400
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'])
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'])


In [54]:
print(1596.7619819523752-396.7619819523753)
print(2796.761981952375-1596.7619819523752)

1200.0
1200.0


In [18]:
from json import load, dump

with open("WesternDietMicrobiomes/oneRXN_localized.json", "r") as jsonIn:
    escherMap = load(jsonIn)

with open("WesternDietMicrobiomes/oneRXN_localized2.json", "w") as jsonOut:
    escherMap = dump(escherMap, jsonOut, indent=3)

# process the ASVMetaboliteInteractions community interactions

## parse the stoichiometric relationships

In [3]:
from pandas import read_csv
from json import dump

interactions = read_csv("ASVMetaboliteInteractions.csv").set_index("ASV")
stoich_mapping = {}
for index, row in interactions.iterrows():
    stoich_mapping[index] = {"reactants":{}, "products":{}}
    for col, ele in row.items():
        metID = col.split("_")[0]
        if ele > 0:   stoich_mapping[index]["products"][metID] = ele
        elif ele < 0:   stoich_mapping[index]["reactants"][metID] = ele

# display(stoich_mapping)
with open("ASV_metabolic_activity_stoich.json", 'w') as jsonOut:
    dump(stoich_mapping, jsonOut, indent=3)

## process the stoich into Escher mapping coordinates